In [1]:
import cdsapi
#url: https://cds.climate.copernicus.eu/api/v2
#key: 159276:b225e1d7-1242-4152-8576-ce50852fbd4e

c = cdsapi.Client()

c.retrieve(
    'reanalysis-era5-single-levels',
    {
        'product_type': 'reanalysis',
        'variable': [
            'soil_temperature_level_1', 'total_precipitation',
        ],
        'year': [
            '2019',
        ],
        'month': [
            '04','05','06',
            '07', '08', '09',
            '10', '11',
        ],
        'day': [
            '01', '02', '03',
            '04', '05', '06',
            '07', '08', '09',
            '10', '11', '12',
            '13', '14', '15',
            '16', '17', '18',
            '19', '20', '21',
            '22', '23', '24',
            '25', '26', '27',
            '28', '29', '30',
            '31', 
        ],
        'time': [
            '06:00', '07:00', '08:00',
            '09:00', '10:00', '11:00',
            '12:00', '13:00', '14:00',
            '15:00', '16:00', '17:00',
            '18:00',
            #13
        ],
        'area': [
            
            38.1445082027146, -97.72654627101196, 38.142173460759004,
            -97.72096882266754,
        ],
        'format': 'netcdf',
    },
    'download.nc')

2022-10-26 16:19:26,942 INFO Welcome to the CDS
2022-10-26 16:19:26,943 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2022-10-26 16:19:27,162 INFO Request is completed
2022-10-26 16:19:27,164 INFO Downloading https://download-0014-clone.copernicus-climate.eu/cache-compute-0014/cache/data2/adaptor.mars.internal-1666817498.425104-20916-13-a7a8205e-3265-480f-87d3-1fe0de4d6e46.nc to download.nc (26.1K)
2022-10-26 16:19:28,411 INFO Download rate 21K/s    


Result(content_length=26748,content_type=application/x-netcdf,location=https://download-0014-clone.copernicus-climate.eu/cache-compute-0014/cache/data2/adaptor.mars.internal-1666817498.425104-20916-13-a7a8205e-3265-480f-87d3-1fe0de4d6e46.nc)

In [29]:
import xarray as xr
#Abrimos la informacion
data_tp=xr.open_dataset('download.nc')

xarray.core.dataset.Dataset

In [33]:
data_tp

<xarray.Dataset>
Dimensions:    (longitude: 1, latitude: 1, time: 3172)
Coordinates:
  * longitude  (longitude) float32 -97.73
  * latitude   (latitude) float32 38.14
  * time       (time) datetime64[ns] 2019-04-01T06:00:00 ... 2019-11-30T18:00:00
Data variables:
    stl1       (time, latitude, longitude) float32 276.8 277.2 ... 281.5 282.5
    tp         (time, latitude, longitude) float32 ...
Attributes:
    Conventions:  CF-1.6
    history:      2022-10-26 20:51:53 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

In [107]:
from datetime import datetime, timedelta
from calendar import monthrange

In [128]:
8*31*13

3224

In [127]:
len(data_tp['stl1'].values.ravel())

3172

In [146]:
year_2019 = {}
year = 2019

# Temperature
t = data_tp['stl1'].values.ravel()
# Precipitation
p = data_tp['tp'].values.ravel()


# Ordered month
months = list(set([x.to_pydatetime().month for x  in data_tp['time'].to_series()]))

for month in sorted(months):
    # Get all ther unique dates.
    # days = set([x.to_pydatetime().day for x  in data_tp['time'].to_series()])

    # 
    month_range = monthrange(year, month)[1]

    days = [x + 1 for x in range(month_range)]
    # number of hours
    n_hours = len(set([x.to_pydatetime().hour for x  in data_tp['time'].to_series()]))

    month_temp = {}

    # Get the temp.
    for day in days:
        day_t = {'temp' : t[0:n_hours]}
        day_p = {'pre' : p[0:n_hours]}
        t = t[n_hours:]
        p = p[n_hours:]

        month_temp.update(
            {day : day_t}
        )

    year_2019.update({
        month : month_temp
    })

year_2019

TypeError: unhashable type: 'dict'

In [145]:
for key, value in year_2019.items():
    print(key, value)

4 {1: {'temp': array([276.7997 , 277.23633, 276.80035, 275.9194 , 275.20642, 275.1801 ,
       275.21924, 275.49557, 276.3836 , 278.6238 , 280.27097, 281.5764 ,
       282.44708], dtype=float32)}, 2: {'temp': array([279.30023, 279.38553, 279.26434, 278.61548, 277.63577, 277.54602,
       277.49472, 277.20877, 278.18527, 280.28635, 281.2872 , 282.82983,
       284.05   ], dtype=float32)}, 3: {'temp': array([281.8713 , 281.61484, 281.27374, 280.63385, 280.3665 , 280.74222,
       281.22693, 281.4071 , 281.9162 , 284.0288 , 284.6892 , 285.7452 ,
       286.45114], dtype=float32)}, 4: {'temp': array([284.795  , 284.40134, 284.13782, 284.03265, 283.8044 , 283.81082,
       283.84418, 283.91278, 283.63065, 284.92966, 286.32803, 287.54175,
       287.68088], dtype=float32)}, 5: {'temp': array([283.72235, 283.08246, 283.08694, 282.5644 , 281.93414, 282.35794,
       282.3432 , 281.9553 , 282.22458, 284.39108, 286.01385, 288.29318,
       289.79993], dtype=float32)}, 6: {'temp': array([287.2833

In [98]:
data_tp['stl1'].values.ravel()

array([276.7997 , 277.23633, 276.80035, ..., 279.72342, 281.46994,
       282.47784], dtype=float32)

In [122]:
[0, 1, 2, 3 , 4 , 5][2:]

[2, 3, 4, 5]

In [118]:
len(set([x.to_pydatetime().hour for x  in data_tp['time'].to_series()]))

13

In [120]:
data_tp['time'].to_series()[0:13]

time
2019-04-01 06:00:00   2019-04-01 06:00:00
2019-04-01 07:00:00   2019-04-01 07:00:00
2019-04-01 08:00:00   2019-04-01 08:00:00
2019-04-01 09:00:00   2019-04-01 09:00:00
2019-04-01 10:00:00   2019-04-01 10:00:00
2019-04-01 11:00:00   2019-04-01 11:00:00
2019-04-01 12:00:00   2019-04-01 12:00:00
2019-04-01 13:00:00   2019-04-01 13:00:00
2019-04-01 14:00:00   2019-04-01 14:00:00
2019-04-01 15:00:00   2019-04-01 15:00:00
2019-04-01 16:00:00   2019-04-01 16:00:00
2019-04-01 17:00:00   2019-04-01 17:00:00
2019-04-01 18:00:00   2019-04-01 18:00:00
Name: time, dtype: datetime64[ns]

(3172, 1, 1)

In [28]:
import json
from statistics import mean
import numpy
#Leemos los datos de temperatura en k° y los almacenamos en una lista
temperature_k = data_tp['stl1'].values.ravel().tolist()
#Leemos los datos de precipitación y los almacenamos en una lista
precipitation_t = data_tp['tp'].values.ravel().tolist()

#convert kelvin to celsius
temperature_celsius = []
for temperature in temperature_k:
  temperature_c= temperature - 273.15
  temperature_celsius.append(temperature_c)
temperature_celsius

[3.6497131347656477,
 4.086328125000023,
 3.6503540039062727,
 2.7694030761718977,
 2.0564208984375227,
 2.0301147460937727,
 2.0692382812500227,
 2.3455749511718977,
 3.2336059570312727,
 5.473809814453148,
 7.120965576171898,
 8.426385498046898,
 9.297082519531273,
 6.150231933593773,
 6.235528564453148,
 6.114343261718773,
 5.465478515625023,
 4.485772705078148,
 4.396020507812523,
 4.344720458984398,
 4.058770751953148,
 5.035272216796898,
 7.136346435546898,
 8.137200927734398,
 9.679833984375023,
 10.899987792968773,
 8.721307373046898,
 8.464837646484398,
 8.123742675781273,
 7.483850097656273,
 7.216485595703148,
 7.592218017578148,
 8.076928710937523,
 8.257104492187523,
 8.766198730468773,
 10.878808593750023,
 11.539208984375023,
 12.595208740234398,
 13.301141357421898,
 11.645013427734398,
 11.251336669921898,
 10.987817382812523,
 10.882653808593773,
 10.654412841796898,
 10.660821533203148,
 10.694177246093773,
 10.762780761718773,
 10.480645751953148,
 11.77965698242189

In [ ]:
precipitation_mm = []
for precipitation in precipitation_t:
  precipitation_m= precipitation *1000
  precipitation_mm.append(precipitation_m)
#temperature_celsius
print(min(precipitation_mm))
print(max(precipitation_mm))
#print (min(temperature_celsius))
#print (max(temperature_celsius))

In [4]:


d = {
  "Temperatura" : temperature_celsius,
  "Precipitacion" : precipitation_mm
}

In [26]:
jsonStr = json.dumps(d)

In [23]:
#Creación de diccionario de fechas
dias=['01', '02', '03','04', '05', '06','07', '08', '09','10', '11', '12','13', '14', '15','16', '17', '18','19', '20', '21','22', '23', '24','25', '26', '27','28', '29', '30']
horas=['06:00', '07:00', '08:00','09:00', '10:00', '11:00','12:00', '13:00', '14:00','15:00', '16:00', '17:00','18:00']
meses= ['April','May','June','July','August','September','October','November']

In [25]:
as1 = {
"Meses"
}
jsonStr = json.dumps(as1)
#jsonStr

SyntaxError: invalid syntax (3460392548.py, line 2)